# Connect to microsoft sql db (MSSQL)

In [ ]:
# First, install the required package if you haven't already
# pip install pyodbc
# Install sqlalchemy if you haven't already

import pandas as pd
from sqlalchemy import create_engine, text
import urllib

# Create connection string for SQLAlchemy
params = urllib.parse.quote_plus(
    'DRIVER={SQL Server};'
    'SERVER=localhost\\SQLEXPRESS;'
    'Trusted_Connection=yes;'
    'DATABASE=TestDB;'  # Specify database directly in connection
)

# Create SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

with engine.connect() as connection:
    customers = pd.read_sql_query(text('SELECT * FROM Customers'), connection)

# Gather and save video game data from unofficial overwatch API 

In [ ]:
import requests 
import json
import os

platform = "pc"
user = "wildwolf-199415"

url = f"https://ow-api.com/v1/stats/{platform}/us/{user}/complete"

# Send the HTTP GET request to the API and parse the JSON response
response = requests.get(url)
data = response.json()

save_folder = r"C:\Users\Ryan\Coding Projects\SQL"
file_path = os.path.join(save_folder, 'overwatch_db.json')

# Writing the json file 
with open(file_path, 'w') as f:
    json.dump(data, f, indent=4)

# Opening the json file
with open(file_path, 'r') as f:
    overwatch_db = json.load(f)

# Postgres SQL

## Read in Overwatch db from local json

In [ ]:
import json

file_path = r"C:\Users\Ryan\Coding Projects\SQL Prac\overwatch_db.json"

with open(file_path, 'r') as f:
    overwatch_db = json.load(f)

In [ ]:
overwatch_db["competitiveStats"]["topHeroes"]

## Defining pandas df to postgressql function

In [11]:
# Actual function saving

from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd

def df_to_postgresql(df: pd.DataFrame, name: str, schema: str, conn_str: str) -> None:

    # Input validation for function
    if not isinstance(df, pd.DataFrame):
        raise TypeError("df must be a pandas DataFrame")
    if not isinstance(name, str):
        raise TypeError("name must be a string")
    if not isinstance(schema, str):
        raise TypeError("schema must be a string")

    # Load variables from .env file
    load_dotenv()
    
    # Create an SQLAlchemy engine
    engine = create_engine(os.getenv(conn_str))

    # name parameter sets the table name, Schema directs to db schema (must be made first in sql)
    df.to_sql(name=name, con=engine, schema=schema, if_exists='replace', index=True)

    # After using the engine to interact with the database
    engine.dispose()

### Saving QP top heros to db 

In [6]:
# Data cleaning 
temp_json = overwatch_db['quickPlayStats']['topHeroes']

QP_Top_Heros = pd.DataFrame(temp_json)

# Simple transpose to flip columns and rows
QP_Top_Heros =  QP_Top_Heros.T

# Putting the index as a regular column in DF 
QP_Top_Heros = QP_Top_Heros.reset_index()
# Renaming that index column to hero
QP_Top_Heros.rename(columns={'index': 'Hero'}, inplace=True)

del temp_json

In [7]:
# Saving the QP_Top_Heros df to an postgres db
df_to_postgresql(QP_Top_Heros, "top_heros", "qp", "postgres_cs_ow")

### Saving QP career stats df to postgres db 

In [8]:
# Data cleaning and transforming
t_json = overwatch_db['quickPlayStats']['careerStats']

# Changing dict/json to df and transposing it to flip columns and rows
qp_career_stats = pd.DataFrame(t_json).T

del t_json

In [9]:
# Function to extract the dictionary from each column except for "heroSpecific" and make each dict pairing into their own column and row  
def expand_dict(qp_career_stats):

    # Put all column names from df into a list  
    dict_columns = qp_career_stats.columns.to_list()

    # Deleting heroSpecific from the column list (will be it's own db)
    del dict_columns[4]

    for col in dict_columns:
        # Expand each dictionary column by using a pd.Series to create a new row for each dictionary it encounters with the dictionary keys as column names and values as the row data.pandas series
        expanded_cols = qp_career_stats[col].apply(lambda x: pd.Series(x, dtype="object"))
        
        # Rename new columns to include original column name as a prefix
        expanded_cols = expanded_cols.add_prefix(f"{col}_")
        
        # Concatenate the expanded columns with the original DataFrame
        qp_career_stats = pd.concat([qp_career_stats, expanded_cols], axis=1)

        # Drop the original dictionary column
        qp_career_stats = qp_career_stats.drop(columns=[col])
    
    # Putting the index as a regular column in DF 
    qp_career_stats = qp_career_stats.reset_index()
    # Renaming that index column to hero
    qp_career_stats.rename(columns={'index': 'Hero'}, inplace=True)

    # dropping the heroSpecific column to be used in later db (SQL can't handle dictionaries in columns)
    qp_career_stats = qp_career_stats.drop(columns=["heroSpecific"])
    
    return qp_career_stats

qp_career_stats = expand_dict(qp_career_stats)

In [10]:
# Saving Quick play career stats data to db
df_to_postgresql(qp_career_stats, "career_stats", "qp", "postgres_cs_ow")

### Saving heroSpecific df to postgres db

In [11]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os

# Data cleaning
t_json = overwatch_db['quickPlayStats']['careerStats']

# Changing dict/json to df and transposing it to flip columns and rows
qp_career_stats = pd.DataFrame(t_json).T

# Putting the index as a regular column in DF 
qp_career_stats = qp_career_stats.reset_index()

# Renaming that index column to hero
qp_career_stats.rename(columns={'index': 'Hero'}, inplace=True)

# Getting each hero and their specific stats into a df, getting rid of first observation because that's all heros
hero_specific = qp_career_stats[['Hero', 'heroSpecific']].iloc[1:]

del qp_career_stats, t_json

In [14]:
# This loop puts each heros specific stats into a unique df named after the hero then saves it postgres
for index, row in hero_specific.iterrows():
    hero = row['Hero']
    hero_dict = row['heroSpecific']

    # Convert the dictionary into a new DataFrame
    new_df = pd.DataFrame([hero_dict])

    # Saves each newly created hero df to postgres db
    df_to_postgresql(new_df, hero, "qp", "postgres_cs_ow")

    # Not best practice but I included the engine.dispose() above so it keeps connecting and disconnecting for each hero entry

## Reading in postgres db to pandas df
If schema or table name is case sensitive, need to put "" around it!!!!!

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

load_dotenv()

# Create an SQLAlchemy engine, make sure to update env file with correct db 
engine = create_engine(os.getenv("postgres_cs_ow"))

# read_sql_query simple 
with engine.connect() as connection:
    df = pd.read_sql_query(
        text('SELECT * FROM qp.ana'),
        connection)

# Using specific columns and conditions
with engine.connect() as connection:
    query = text("""
        SELECT card_number, name, color, print_completed
        FROM archive
        WHERE color = :color""")
    
    df_filtered = pd.read_sql_query(
        query,
        connection,
        params={"color": "Blue"})

engine.dispose()

## Read, modify local, Update SQL db (3D printing workflow example)

Hypothetical: read in data from sql, then execute patron_contacting email script from 3d printing, results in new df called df_filtered, update existing sql db with df_filtered 

In [14]:
# Step 1 - Read in data from sql with specific paramters

import pandas as pd
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Loading keys in
load_dotenv()

# Create an SQLAlchemy engine
engine = create_engine(os.getenv("postgres_cs_3d"))

# read_sql_query simple 
with engine.connect() as connection:
    df_filter = pd.read_sql_query(
        text("""SELECT * FROM archive
                WHERE print_completed = 'X'
                AND patron_contacted IS NULL
                AND invalid_email IS NULL
                AND picked_up IS NULL"""),
        connection)

In [ ]:
# Step 2: Mimick sending out emails to patrons
# Results in modified df -> df_filter

import datetime as dt
# Assuming `df_filter` is a pandas DataFrame
df_filter.loc[df_filter["print_completed"] == "X", "patron_contacted"] = "X"

today_str = dt.date.today().strftime("%m/%d/%y") 
today_date = dt.datetime.strptime(today_str, "%m/%d/%y").date()
df_filter.loc[df_filter["print_completed"] == "X", "contacted_date"] = today_date

In [ ]:
# Step 3: Update Postgres db based on new inputs from df_filtered 

# Update query
update_query = """
    UPDATE archive 
    SET patron_contacted = :patron_contacted,
        contacted_date = :contacted_date,
        invalid_email = :invalid_email
    WHERE card_number = :card_number
"""

# Convert DataFrame rows to list of dictionaries for batch update (connection.execute used dictionaries as input for the parameters function)
records_to_update = df_filter.to_dict('records')

# Establish connection and execute updates
with engine.begin() as connection:
    try:
        for record in records_to_update:
            connection.execute(
                text(update_query),
                parameters=record)

        print(f"Successfully updated {len(records_to_update)} records in the database")
        
    except Exception as e:
        print(f"Error updating database: {str(e)}")
        connection.rollback() # If an error occurs during the database update process, any partial or uncommitted changes made during the transaction are undone.
        raise

# Release and clean up all database connections managed by the SQLAlchemy engine 
engine.dispose()

## MYSQL db 

In [ ]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import pandas as pd
import os 
from dotenv import load_dotenv

def load_mysql_db(db): 

    load_dotenv()

    # Define your credentials
    username = 'root'
    password = quote_plus(os.getenv("mysql_pass"))  # Encodes the special characters
    host = 'localhost' 
    port = 3306
    database = db

    # Create the connection string
    connection_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"

    # Create the SQLAlchemy engine
    engine = create_engine(connection_string)

    # Test the connection
    try:
        with engine.connect() as connection:
            # Use pd.read_sql_query to fetch data
            df = pd.read_sql_query(text("SELECT * FROM city"), connection)
            print(df)
    except Exception as e:
        print(f"Connection failed: {e}")
    
    return df


df = load_mysql_db("sakila")

In [ ]:
# Inferior way to connecting to mysql
import pymysql

conn = pymysql.connect(
    host = "localhost",
    user = "root",
    password = os.getenv("mysql_pass") ,
    database = "sakila"
)

cursor = conn.cursor()
cursor.execute("SELECT * FROM city")
results = cursor.fetchall()
print(results)

cursor.close()
conn.close() 

# Using pandas read_sql function is untested outside SQLAlchemy, so it'll throw a warning but still work
# df = pd.read_sql("SELECT * FROM city", conn)

# SQLite

In [ ]:
from dotenv import load_dotenv
import sqlite3
import pandas as pd
import os

load_dotenv()

postgres_engine = create_engine(os.getenv("postgres_cs_nba"))

# SQLite connection
sqlite_file  = "nba_stats.sqlite"

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_file)

# Query to select a table
table_name = 'common_player_info'
query = f"SELECT * FROM {table_name}"

# Load the table into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
print(df)

# Random excel code

In [13]:
prints = pd.read_excel(r"C:\Users\Ryan\Desktop\EGR KDL Master 3D Printing List.xlsx")

df_to_postgresql(prints, "archive", "public", "postgres_cs_3d")

In [ ]:
directory_save = (r"C:\Users\Ryan\Desktop")
file_name = f"qp_career_stats.xlsx" 
file_path = os.path.join(directory_save, file_name) 

qp_career_stats.to_excel(file_path , index=True)

sqlalchemy + pandas is the way to go for interacting with any SQL db then modifying it with pandas    
psycopg2=postgres  &  pymysql=mysql